# NV Saturation measurement

Required modules
- powermeterlogic
- laserlogic

In [1]:
import os
import time
import matplotlib.pyplot as plt

from collections import OrderedDict
import datetime

from logic.generic_logic import GenericLogic

In [2]:
class SaturationLogic(GenericLogic):
    def __init__(self):
        self.count_freq = counterlogic._count_frequency
        self.laserlogic = laserlogic
        self.powermeterlogic = powermeterlogic
        self.counterlogic = counterlogic
        self.savelogic = savelogic
        
        self.data = {}
        self.start_power = 0
        self.final_power = 0
        self.num_points = 0
        self.counts_average_from_end = 0
        self.measurement_start_time = None
        self.measurement_stop_time = None
        
    def draw_figure(self):
        powers, counts = self.data["Powermeter power (W)"], self.data["Avg. counts"]

        plt.style.use(savelogic.mpl_qudihira_style)

        fig, ax = plt.subplots()

        ax.errorbar(powers * 1e6, counts, yerr=self.data["Count errors"], capsize=3, fmt="o-")
        ax.set_xlabel("Powermeter power (uW)")
        ax.set_ylabel("Mean counts (kcps)")
        
        fig.tight_layout()
        return fig

    def save_data(self, tag=None):
        timestamp = datetime.datetime.now()

        parameters = OrderedDict()
        
        data_to_save = OrderedDict()
        data_to_save["Laser power (W)"] = self.data["Laser power (W)"]
        data_to_save["Powermeter power (W)"] = self.data["Powermeter power (W)"]
        data_to_save["UHV objective power (W)"] = self.data["UHV objective power (W)"]
        data_to_save["RT objective power (W)"] = self.data["RT objective power (W)"]
        data_to_save["Avg. counts"] = self.data["Avg. counts"]
        data_to_save["Upper count errors"] = self.data["Count errors"][0, :]
        data_to_save["Lower count errors"] = self.data["Count errors"][1, :]

        filepath = self.savelogic.get_path_for_module(module_name="Saturation")

        if tag:
            filelabel = f"{tag}_saturation"
        else:
            filelabel = "saturation"
        
        self.log.info('Saturation saved to:\n{0}'.format(filepath))
        print('Saturation saved to:\n{0}'.format(filepath))

        fig = self.draw_figure()

        self.savelogic.save_data(data_to_save, filepath=filepath, parameters=parameters, filelabel=filelabel, 
                                 timestamp=timestamp, plotfig=fig, delimiter='\t')
        
    def set_up_measure(self, start_power, final_power, num_points, counts_average_from_end):
        self.start_power = start_power
        self.final_power = final_power
        self.num_points = num_points
        self.counts_average_from_end = counts_average_from_end
        
    def start_measure(self):
        self.data = {}
        laser_powers = np.linspace(self.start_power, self.final_power, self.num_points)
        counts = np.zeros(self.num_points, dtype=int)
        powermeter_powers = np.zeros(self.num_points)
        count_errors = np.zeros((2, self.num_points))
        
        current_power = self.laserlogic.laser_power
        
        self.measurement_start_time = datetime.datetime.now()

        for idx, laser_power in enumerate(laser_powers):
            try: 
                self.laserlogic.set_power(laser_power)
            except ValueError:
                time.sleep(1)
                self.laserlogic.set_power(laser_power)
            
            # Sleep for 5s
            time.sleep(150 / self.count_freq)
            self.powermeterlogic.get_power(state=None)
            powermeter_powers[idx] = self.powermeterlogic.power
            
            count_array = self.counterlogic.countdata_smoothed[0, -self.counts_average_from_end:]
            counts[idx] = np.mean(count_array)
            count_errors[0, idx] = np.max(count_array) - counts[idx]
            count_errors[1, idx] = counts[idx] - np.min(count_array)

            print(f"[{idx+1}/{len(laser_powers)}] laser power={laser_power * 1e3} mW, powermeter power={powermeter_powers[idx] * 1e6:.2f} uW, counts={counts[idx]}")
        
        self.measurement_stop_time = datetime.datetime.now()

        self.data = {
            "Laser power (W)": laser_powers, 
            "Powermeter power (W)": powermeter_powers, 
            "UHV objective power (W)": powermeter_powers * powermeterlogic.calibration_param_uhv, 
            "RT objective power (W)": powermeter_powers * powermeterlogic.calibration_param_rt,
            "Avg. counts": counts,
            "Count errors": count_errors
        }

In [4]:
sat = SaturationLogic()
sat.set_up_measure(start_power=5e-3, final_power=100e-3, num_points=20, counts_average_from_end=75)
sat.start_measure()
sat.save_data(tag="NV_with_90_10_BS")

[1/20] laser power=5.0 mW, powermeter power=0.02 uW, counts=357
[2/20] laser power=10.0 mW, powermeter power=0.78 uW, counts=3632
[3/20] laser power=15.0 mW, powermeter power=2.15 uW, counts=7770
[4/20] laser power=20.0 mW, powermeter power=4.66 uW, counts=14571
[5/20] laser power=25.0 mW, powermeter power=8.99 uW, counts=22219
[6/20] laser power=30.000000000000004 mW, powermeter power=12.80 uW, counts=27360
[7/20] laser power=34.99999999999999 mW, powermeter power=16.50 uW, counts=31010
[8/20] laser power=40.0 mW, powermeter power=19.78 uW, counts=33809
[9/20] laser power=45.0 mW, powermeter power=23.59 uW, counts=37242
[10/20] laser power=49.99999999999999 mW, powermeter power=28.44 uW, counts=39926
[11/20] laser power=55.0 mW, powermeter power=33.24 uW, counts=41999
[12/20] laser power=60.0 mW, powermeter power=39.11 uW, counts=43861
[13/20] laser power=65.0 mW, powermeter power=44.52 uW, counts=45394
[14/20] laser power=70.0 mW, powermeter power=51.10 uW, counts=47184
[15/20] laser